引入头文件

In [10]:
import random
import numpy as np
import cv2
import os
import math
import time

generate函数作用：生成光斑，可以通过其获取S集合

In [42]:
# def generate(width, height, num, gap, seed):
#     random.seed(seed)
    
#     point_index = np.arange(width * height)
#     unused_point_set = np.arange(width * height)
#     unused_point_num = width * height

#     used_point_set = []
#     used_point_num = 0
#     while used_point_num != num and unused_point_num != 0:
#         rd = random.randint(0, unused_point_num - 1)
#         x = unused_point_set[rd] % width
#         y = unused_point_set[rd] / width
#         used_point_set.append((x, y))
#         used_point_num += 1
#         unused_point_set[rd] = unused_point_set[unused_point_num - 1]
#         point_index[unused_point_set[unused_point_num - 1]] = rd
#         unused_point_num -= 1

#         # image = np.zeros((height, width), np.uint8)

#         for i in range(x - gap, x + gap + 1):
#             h = int(math.floor((gap ** 2 - (x - i) ** 2) ** .5))
#             ii = i % width
#             for j in range(y - h, y + h + 1):
#                 jj = j % height
#                 # image[jj, ii] = 255
#                 index = jj * width + ii
#                 if point_index[index] < unused_point_num:
#                     unused_point_set[point_index[index]] = unused_point_set[unused_point_num - 1]
#                     point_index[unused_point_set[unused_point_num - 1]] = point_index[index]
#                     unused_point_num -= 1

#         cv2.imshow('hehe', image)
#         cv2.waitKey()

#     print(used_point_num)

#     return used_point_set

In [112]:
def generate(width, height, num, gap, seed):
    random.seed(seed)
    
    point_index = np.arange(width * height)
    unused_point_set = np.arange(width * height)
    unused_point_num = width * height

    used_point_set = []
    used_point_num = 0
    while used_point_num != num and unused_point_num != 0:
        rd = random.randint(0, unused_point_num - 1)
        x = unused_point_set[rd] % width
        y = unused_point_set[rd] / width
        used_point_set.append((x, y))
        used_point_num += 1
        point = unused_point_set[rd]
        unused_point_set[rd] = unused_point_set[unused_point_num - 1]
        point_index[unused_point_set[unused_point_num - 1]] = rd
        point_index[point] = -1
        unused_point_num -= 1


        for i in range(x - gap, x + gap + 1):
            h = int(math.floor((gap ** 2 - (x - i) ** 2) ** .5))
            ii = i % width
            for j in range(y - h, y + h + 1):
                jj = j % height
                index = jj * width + ii
                if point_index[index] >= 0:
                    unused_point_set[point_index[index]] = unused_point_set[unused_point_num - 1]
                    point_index[unused_point_set[unused_point_num - 1]] = point_index[index]
                    point_index[index] = -1
                    unused_point_num -= 1

    return used_point_set

def generate_2(width, height, num, gap, seed):
    
    random.seed(seed)
    
    point_index = np.arange(width * height)
    unused_point_set = np.arange(width * height)
    unused_point_num = width * height

    margin_index = np.zeros((width * height), np.int32) - 1
    margin_set = np.zeros((width * height), np.int32)
    margin_num = 0

    used_point_set = []
    used_point_num = 0

    rd = random.randint(0, unused_point_num - 1)
    margin_set[margin_num] = unused_point_set[rd]
    margin_index[unused_point_set[rd]] = margin_num
    margin_num += 1
    point = unused_point_set[rd]
    unused_point_set[rd] = unused_point_set[unused_point_num - 1]
    point_index[unused_point_set[unused_point_num - 1]] = rd
    point_index[point] = -1
    unused_point_num -= 1

    while used_point_num != num and margin_num != 0:
        rd = random.randint(0, margin_num - 1)
        x = margin_set[rd] % width
        y = margin_set[rd] / width
        used_point_set.append((x, y))
        used_point_num += 1

        point = margin_set[rd]
        margin_set[rd] = margin_set[margin_num - 1]
        margin_index[margin_set[margin_num - 1]] = rd
        margin_index[point] = -1
        margin_num -= 1

        gap_out = gap + 1

        for i in range(x - int(math.ceil(gap_out - 1)), x + int(math.ceil(gap_out - 1)) + 1):
            h_out = int(math.ceil((gap_out ** 2 - (x - i) ** 2) ** .5 - 1))
            h_in = int(math.ceil((gap ** 2 - (x - i) ** 2) ** .5 - 1))
            ii = i % width
            for j in range(int(y - h_out), int(y + h_out + 1)):
                jj = j % height
                index = jj * width + ii

                is_margin = False
                if j < y - h_in or j > y + h_in:
                    is_margin = True

                if is_margin:
                    if point_index[index] >= 0:
                        margin_set[margin_num] = index
                        margin_index[index] = margin_num
                        margin_num += 1


                else:
                    if margin_index[index] >= 0:
                        margin_set[margin_index[index]] = margin_set[margin_num - 1]
                        margin_index[margin_set[margin_num - 1]] = margin_index[index]
                        margin_index[index] = -1
                        margin_num -= 1

                if point_index[index] >= 0:
                    unused_point_set[point_index[index]] = unused_point_set[unused_point_num - 1]
                    point_index[unused_point_set[unused_point_num - 1]] = point_index[index]
                    point_index[index] = -1
                    unused_point_num -= 1

    return used_point_set

In [113]:
generate_2(1080, 1080, 100000, 11, 1)

[(982, 260.90925925925927),
 (979, 249.90648148148148),
 (973, 239.90092592592592),
 (965, 231.8935185185185),
 (960, 222.88888888888889),
 (960, 212.88888888888889),
 (949, 219.87870370370371),
 (969, 255.89722222222221),
 (970, 207.89814814814815),
 (990, 249.91666666666666),
 (984, 239.9111111111111),
 (975, 217.90277777777777),
 (1000, 245.92592592592592),
 (960, 201.88888888888889),
 (939, 215.86944444444444),
 (1010, 251.93518518518519),
 (1021, 250.94537037037037),
 (1023, 240.94722222222222),
 (970, 266.89814814814815),
 (933, 205.86388888888888),
 (941, 198.87129629629629),
 (985, 222.91203703703704),
 (1015, 261.93981481481484),
 (1029, 230.95277777777778),
 (962, 243.89074074074074),
 (1008, 238.93333333333334),
 (1036, 222.95925925925926),
 (990, 212.91666666666666),
 (947, 188.87685185185185),
 (977, 275.90462962962965),
 (973, 286.90092592592595),
 (926, 196.85740740740741),
 (987, 281.91388888888889),
 (959, 267.88796296296294),
 (1021, 271.94537037037037),
 (970, 195.89

首先，获取集合S
N = 集合当中光斑的数目
height = 光斑的高度 width = 光斑的宽度
W = 获取到光斑的维度 = height * width
b = 分类的节点数目
Z = 表征权重集合，维度为w * b
sita = 常数值
k = 设置系数Z的稀疏程度
greedy_range = z中值的取值范围， 此处可尝试设置为[-1, 1]
step = 0.1, 设置z中数值的取值间隔


In [47]:
height = 11
width = 11
N = 9* 10 ** 5
W = height * width
b = 32
k = 5
step = 0.1
times = 10



此处设置Node

In [48]:
class Node(object):
    def __init__(self, S):
        self.W_S, self.N_S = S.shape
        self.S = S
        seed = time.time()
        self.leftnode = None
        self.rightnode = None
        self.prefect_z = None
        self.prefect_sita = None
        self.prefect_right_S = None
        self.prefect_left_S = None
        self.leaf = False
        np.random.seed(int(time.time()))
       
    
    def find_the_biggest_information_gain(self):
        
        HS = self.entropy(self.S)
        # print('当前节点的总熵为' + str(HS))
        info_gain = -999.0
        
        for i in range(times):
#             start = time.time()
            if i % 5 == 0:
                print("当前运行到%d" % (i))
            z = np.random.rand(W, 1) * 2 -1
#             sita = np.random.rand(self.N_S,1) * 2 -1
            sita = np.random.rand()*2-1
            result = np.dot(self.S.T, z) + sita
            cnt_P = 0
            cnt_N = 0
            P_index = []
            N_index = []
            
#             P_index = [ii for ii in range(self.N_S) if result[ii][0] >= 0]
#             cnt_P = len(P_index)
#             N_index = [ii for ii in range(self.N_S) if result[ii][0] < 0]
#             cnt_N = len(N_index)

            for ii in range(self.N_S):
                if result[ii][0] >= 0:
                    P_index.append(ii)
#                     cnt_P += 1
                else:
                    N_index.append(ii)
#                     cnt_N += 1
            cnt_N = len(N_index)
            cnt_P = len(P_index)
            pos_S = self.S[:, P_index]
            neg_S = self.S[:, N_index]
#             if cnt_N == 0 or cnt_P ==0:
#                 continue
#             print('进行了一次分类，然后将分类结果求解信息增益' + str(neg_S.shape) +str(pos_S.shape))
            ans = self.information_gain(cnt_P, cnt_N, pos_S, neg_S)
            mid_v = HS - ans 
            if mid_v > info_gain: 
                info_gain = mid_v
                self.prefect_z = z
                self.prefect_sita = sita
                self.prefect_right_S  = pos_S
                self.prefect_left_S = neg_S
#             end = time.time()
#             print('共用时%d' % (end - start))
        print('已经完成10次随机尝试，选出了当前最好效果的z和sita' + str(sita))
        if self.prefect_right_S.shape[1] and self.prefect_left_S.shape[1]:
            new_node_right = Node(self.prefect_right_S)
            print('建立右节点 ' + '右节点数据为' + str(self.prefect_right_S.shape))
            self.rightnode = new_node_right
            new_node_left = Node(self.prefect_left_S)
            self.leftnode = new_node_left
            print('建立左节点 ' + '左节点数据为' + str(self.prefect_left_S.shape))
        else:
            self.leaf = True
            
    
    def information_gain(self, cnt_P, cnt_N, pos_S, neg_S):
        return float(cnt_N / self.N_S) *self.entropy(neg_S) + float(cnt_P / self.N_S) * self.entropy(pos_S) 
           
    def entropy(self, s):
#         print('要进行协方差运算的shape' + str(s.shape))
        con_arr = np.cov(s)
#         print('协方差运算结束之后的shape' + str(con_arr.shape))
        det = float(np.linalg.det(con_arr))
        # print('行列式的值为' + str(det))
#         if det <= 0:
#             det = 0.1 ** 6
        try:
            Hs = 0.5 * W* math.log(2 * math.pi * math.e) + 0.5 * math.log(det)
        except:
            print('det is' + str(det))
            Hs = 0.5 * W* math.log(2 * math.pi * math.e) + 0.5 # 将det 设置增大 然后把这个结果去掉（当前还有隐患）
#         if np.isnan(Hs):
#             Hs = 0
#         assert Hs >= 0
        return Hs  
    


上部分已经完成了节点类的声明，下一步进行建树操作。

In [49]:
def build_tree(root, height):
    if height == 5:
        return
    root.find_the_biggest_information_gain()
    
    if root.leftnode != None and root.rightnode != None:
        build_tree(root.leftnode, height + 1)
        build_tree(root.rightnode, height + 1)
        

随机设置一些样本进行测试,设立前序遍历函数，查看排序的结果

In [50]:
def travel_tree(root):
    if root.leftnode!=None:
        travel_tree(root.leftnode)
    print(root.prefect_z)
    print(root.leaf)
    print('\n')
    if root.rightnode!=None:
        travel_tree(root.rightnode)

In [51]:
test_S = np.random.randint(0, 2,(121, 900000))
# print(test_S)
# print(test_S.shape)
root = Node(test_S)
build_tree(root, 0)
travel_tree(root)



当前运行到0
当前运行到5
已经完成10次随机尝试，选出了当前最好效果的z和sita-0.4399127777865197
建立右节点 右节点数据为(121, 466518)
建立左节点 左节点数据为(121, 433482)
当前运行到0
当前运行到5
已经完成10次随机尝试，选出了当前最好效果的z和sita-0.8677524765512963
建立右节点 右节点数据为(121, 204820)
建立左节点 左节点数据为(121, 228662)
当前运行到0
当前运行到5
已经完成10次随机尝试，选出了当前最好效果的z和sita-0.15947242296615727
建立右节点 右节点数据为(121, 124319)
建立左节点 左节点数据为(121, 104343)
当前运行到0
当前运行到5
已经完成10次随机尝试，选出了当前最好效果的z和sita-0.00836675121127417
建立右节点 右节点数据为(121, 54549)
建立左节点 左节点数据为(121, 49794)
当前运行到0
当前运行到5
已经完成10次随机尝试，选出了当前最好效果的z和sita0.05553244259585366
建立右节点 右节点数据为(121, 26855)
建立左节点 左节点数据为(121, 22939)
当前运行到0
当前运行到5
已经完成10次随机尝试，选出了当前最好效果的z和sita-0.45822729252956673
建立右节点 右节点数据为(121, 28043)
建立左节点 左节点数据为(121, 26506)
当前运行到0
当前运行到5
已经完成10次随机尝试，选出了当前最好效果的z和sita0.07302537740873927
建立右节点 右节点数据为(121, 64635)
建立左节点 左节点数据为(121, 59684)
当前运行到0
当前运行到5
已经完成10次随机尝试，选出了当前最好效果的z和sita0.515844705358754
建立右节点 右节点数据为(121, 25734)
建立左节点 左节点数据为(121, 33950)
当前运行到0
当前运行到5
已经完成10次随机尝试，选出了当前最好效果的z和sita0.888184170982766
建立右节点 右节点数据为(121, 33754)
建立左节点 左节点数据为

In [21]:
travel_tree(root)

[[1 1 1 ..., 1 1 1]
 [0 0 1 ..., 1 0 0]
 [0 0 1 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 0 0]
 [1 0 1 ..., 0 1 1]
 [0 0 0 ..., 1 0 1]]




AttributeError: 'NoneType' object has no attribute 'shape'

检验拼接方法的可行性

In [ ]:
a = np.random.rand(3,3)
b = np.random.rand(3,3)
c = [1,2]

print(a)
print(b)
d = a[:,c]
e  = b[:, c]
print(d)
print(type(d),d.shape)
f = np.concatenate((d,e), axis = 1)
print(f)

检验建树方法的可行性

In [37]:
# class node(object):
#     def __init__(self, n):
#         self.v = n
#         self.l = None
#         self.r = None
        
# def build(root, height):
#     if height == 5:
#         return
#     else:
#         l = node(height)
#         r = node(height)
#         root.l = l
#         root.r = r
#         build(root.l, height+1)
#         build(root.r, height + 1)

# def travel(root):
#     if root.l!=None:
#         travel(root.l)
#     print(root.v)
#     if root.r!=None:
#         travel(root.r)
    

# root = node(0)
# build(root,0)
# travel(root)
        
        

4
3
4
2
4
3
4
1
4
3
4
2
4
3
4
0
4
3
4
2
4
3
4
1
4
3
4
2
4
3
4
0
4
3
4
2
4
3
4
1
4
3
4
2
4
3
4
0
4
3
4
2
4
3
4
1
4
3
4
2
4
3
4


In [155]:
num = 90
c = np.zeros((121, num))
for i in range(num):
    a = np.zeros((11, 11))
    a [4:7, 4:7] = 1
    a [0:4, 0:4] = 1
    a [0:4,8:11] = 1
    a = a.reshape(121)
    # print(a.shape)
    c[:,i] = a
np.float64(np.linalg.det(np.cov(c)))

0.0

In [7]:
#!/usr/bin/env python
# encoding: utf-8

import numpy as np
import cv2
import random
import math
import matplotlib.pyplot as plt
import sys

def generate(width, height, num, gap, seed):
    random.seed(seed)
    
    point_index = np.arange(width * height)
    unused_point_set = np.arange(width * height)
    unused_point_num = width * height

    used_point_set = []
    used_point_num = 0
    while used_point_num != num and unused_point_num != 0:
        rd = random.randint(0, unused_point_num - 1)
        x = unused_point_set[rd] % width
        y = int(unused_point_set[rd] / width)
        used_point_set.append((x, y))
        used_point_num += 1
        point = unused_point_set[rd]
        unused_point_set[rd] = unused_point_set[unused_point_num - 1]
        point_index[unused_point_set[unused_point_num - 1]] = rd
        point_index[point] = -1
        unused_point_num -= 1


        for i in range(int(x - gap), int(x + gap + 1)):
            h = int(math.floor((gap ** 2 - (x - i) ** 2) ** .5))
            ii = i % width
            for j in range(int(y - h), int(y + h + 1)):
                jj = j % height
                index = jj * width + ii
                if point_index[index] >= 0:
                    unused_point_set[point_index[index]] = unused_point_set[unused_point_num - 1]
                    point_index[unused_point_set[unused_point_num - 1]] = point_index[index]
                    point_index[index] = -1
                    unused_point_num -= 1

    return used_point_set

def generate_2(width, height, num, gap, seed):
    
    random.seed(seed)
    
    point_index = np.arange(width * height)
    unused_point_set = np.arange(width * height)
    unused_point_num = width * height

    margin_index = np.zeros((width * height), np.int32) - 1
    margin_set = np.zeros((width * height), np.int32)
    margin_num = 0

    used_point_set = []
    used_point_num = 0

    rd = random.randint(0, unused_point_num - 1)
    margin_set[margin_num] = unused_point_set[rd]
    margin_index[unused_point_set[rd]] = margin_num
    margin_num += 1
    point = unused_point_set[rd]
    unused_point_set[rd] = unused_point_set[unused_point_num - 1]
    point_index[unused_point_set[unused_point_num - 1]] = rd
    point_index[point] = -1
    unused_point_num -= 1

    while used_point_num != num and margin_num != 0:
        rd = random.randint(0, margin_num - 1)
        x = margin_set[rd] % width
        y = margin_set[rd] / width
        used_point_set.append((x, y))
        used_point_num += 1

        point = margin_set[rd]
        margin_set[rd] = margin_set[margin_num - 1]
        margin_index[margin_set[margin_num - 1]] = rd
        margin_index[point] = -1
        margin_num -= 1

        gap_out = gap + 1

        for i in range(x - int(math.ceil(gap_out - 1)), x + int(math.ceil(gap_out - 1)) + 1):
            h_out = int(math.ceil((gap_out ** 2 - (x - i) ** 2) ** .5 - 1))
            h_in = int(math.ceil((gap ** 2 - (x - i) ** 2) ** .5 - 1))
            ii = i % width
            for j in range(y - h_out, y + h_out + 1):
                jj = j % height
                index = jj * width + ii

                is_margin = False
                if j < y - h_in or j > y + h_in:
                    is_margin = True

                if is_margin:
                    if point_index[index] >= 0:
                        margin_set[margin_num] = index
                        margin_index[index] = margin_num
                        margin_num += 1


                else:
                    if margin_index[index] >= 0:
                        margin_set[margin_index[index]] = margin_set[margin_num - 1]
                        margin_index[margin_set[margin_num - 1]] = margin_index[index]
                        margin_index[index] = -1
                        margin_num -= 1

                if point_index[index] >= 0:
                    unused_point_set[point_index[index]] = unused_point_set[unused_point_num - 1]
                    point_index[unused_point_set[unused_point_num - 1]] = point_index[index]
                    point_index[index] = -1
                    unused_point_num -= 1

    return used_point_set

def main():

    dropout_ratio = [.0, .1, .2, .4]

    width = 120
    height = 80
    gap = 7
    point_num = 250

    point_radius = 2

    shift = 1

    offset = 20


    image = np.zeros((height, width), np.float32)
    image_fill = np.zeros((height + offset*2, width + offset*2), np.float32)
    scale_factor = 4
    points = generate(width*scale_factor, height*scale_factor, point_num, int(gap*scale_factor-scale_factor/4*1), 8888)
#     print(points)

    display = np.zeros((height*scale_factor, width*scale_factor), np.float32)
    for x, y in points:
        cv2.circle(display, (x, y), int(point_radius*scale_factor*.6), (1,), -1)
#     cv2.imshow('hehe', display)
    train_x= list()
    for x, y in points:
        print(display[x-6:x+5,y-6:y+5].shape)
        try: 
            train_x.append(display[x-6:x+5,y-6:y+5].reshape(121))
        except:
            pass
#     print(len(train_x))
    train_ = np.array(train_x).reshape(len(train_x), 121).T
    print(train_.shape)
#     print(train_)
    return train_
    
    cv2.waitKey()

    points = [(int(round(x/float(scale_factor))), int(round(y/float(scale_factor)))) for x, y in points]
#     print('the shape of img_fill is '+ str(image_fill.shape))

    for p in points:
        
        cv2.circle(image, p, point_radius, (1,), -1)
        x = p[0] + offset
        y = p[1] + offset
#         print('x is %d, y is %d' % (x,y))
        cv2.circle(image_fill, (x, y), point_radius, (1,), -1)
        if x >= width:
            cv2.circle(image_fill, (x-width, y), point_radius, (1,), -1)
        if x < offset*2:
            cv2.circle(image_fill, (x+width, y), point_radius, (1,), -1)
        if y >= height:
            cv2.circle(image_fill, (x, y-height), point_radius, (1,), -1)
        if y < offset*2:
            cv2.circle(image_fill, (x, y+height), point_radius, (1,), -1)
        if x >= width and y >= height:
            cv2.circle(image_fill, (x-width, y-height), point_radius, (1,), -1)
        if x < offset*2 and y < offset*2:
            cv2.circle(image_fill, (x+width, y+height), point_radius, (1,), -1)
        if y >= height and x < offset*2:
            cv2.circle(image_fill, (x+width, y-height), point_radius, (1,), -1)
        if y < offset*2 and x >= width:
            cv2.circle(image_fill, (x-width, y+height), point_radius, (1,), -1)



    cv2.rectangle(image_fill, (offset, offset), (offset + width, offset + height), (64,), 1)
    cv2.imshow('hehe', image_fill)
    cv2.waitKey()



In [8]:
# test_S = np.random.randint(0, 2,(121, 900000))
# print(test_S)
# print(test_S.shape)
test_S = main()
root = Node(test_S)
build_tree(root, 0)
travel_tree(root)



(0, 11)
(0, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(0, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(11, 11)
(0, 11)
(0, 11)
(0, 11)
(0, 11)
(11, 11)
(0, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(0, 11)
(0, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 0)
(11, 11)
(11, 11)
(11, 0)
(11, 0)
(11, 11)
(0, 11)
(11, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 0)
(11, 11)
(0, 11)
(0, 11)
(11, 11)
(0, 11)
(11, 11)
(0, 11)
(11, 11)
(0, 11)
(11, 11)
(6, 11)
(11, 11)
(11, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(0, 11)
(11, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)
(0, 11)
(11, 11)
(11, 11)
(11, 11)
(11, 11)
(0, 11)


E:\Anaconda_3.6\Anaconda\lib\site-packages\ipykernel_launcher.py:78: RuntimeWarning: Degrees of freedom <= 0 for slice
E:\Anaconda_3.6\Anaconda\lib\site-packages\numpy\lib\function_base.py:2929: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)
E:\Anaconda_3.6\Anaconda\lib\site-packages\numpy\lib\function_base.py:2929: RuntimeWarning: invalid value encountered in multiply
  c *= 1. / np.float64(fact)
E:\Anaconda_3.6\Anaconda\lib\site-packages\numpy\linalg\linalg.py:1821: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
E:\Anaconda_3.6\Anaconda\lib\site-packages\numpy\lib\function_base.py:1110: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
E:\Anaconda_3.6\Anaconda\lib\site-packages\numpy\core\_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
已经完成150次随机尝试，选出了当前最好效果的z和sita
建立右节点 右节点数据为(121, 3)
建立左节点 左节点数据为(121, 6)
det is0.0
当前运行到0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
当前运行到5
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
已经完成150次随机尝试，选出了当前最好效果的z和sita
建立右节点 右节点数据为(121, 0)
建立左节点 左节点数据为(121, 6)
det is0.0
当前运行到0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
当前运行到5
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
det is0.0
已经完成150次随机尝试，选出了当前最好效果的z和sita
建立右节点 右节点数据为(121, 0)
建立左节点 左节点数据为(121, 6)
当前运行到0


ZeroDivisionError: division by zero